# Example SciDB queries for trade and quote data

This iPython Notebook replicates the work done by AFL queries at [this repo's main page](https://github.com/Paradigm4/TAQ/) in SciDB-py. 

First follow the first few steps on that page to load trades and quotes data into SciDB. The last command you should run there is:
```
./quotes_redim.sh
```

Then let's run some fun queries on the loaded trades and quote (TAQ) data from the comfort of SciDB-Py.

First let us set up the connectivity to the database (and also import some useful libraries)

In [1]:
import matplotlib.pyplot as plt
from scidbpy import connect
import time
sdb = connect() 
#sdb = connect("http://10.0.20.185:8080")
afl = sdb.afl

Next, associate Python objects with SciDB arrays (no data transfers yet -- just connections)

In [2]:
trades = sdb.wrap_array('trades')
quotes = sdb.wrap_array('quotes')
tkr = sdb.wrap_array('tkr')

An example lookup

In [3]:
trades.todataframe().head()

price  volume  sequence_number condition  \
dummy symbol_index ms                                                    
0     0            34200011  61.000   32051             2325       O     
                   34201498  60.100      30             2971      I      
                   34207432  60.025     100             3782      @      
                   34207506  60.100     100             3785      @      
                   34210008  60.800     100             4112      Q      

                            exchange  
dummy symbol_index ms                 
0     0            34200011        N  
                   34201498        N  
                   34207432        D  
                   34207506        B  
                   34210008        P

## Looking up trades by symbol string

Join with the auxiliary `tkr` array to look up data by ticker symbol name. Here are examples that count the number of trades and quotes for 'BAM' {??}.


In [4]:
symbol = "AA"
print(sdb.merge(trades,
            tkr.filter("symbol='%s'" % symbol)
          ).nonempty())
print(sdb.merge(quotes,
            tkr.filter("symbol='%s'" % symbol)
          ).nonempty())

68817
104040


As expected we see more quotes than trades for this instrument. Note that you can also just filter directly by symbol index using `between` if you know it. For example:

In [5]:
tkr.filter("symbol='%s'" % symbol).todataframe()

,symbol
symbol_index,
1,AA


In [6]:
print(trades.dim_names)
trades.between('null', 1, 'null', 'null', 1, 'null').nonempty()

[u'dummy', u'symbol_index', u'ms']


68817

## Computing minute bars

The trade data are now organized by symbol, time, and a dummy coordinate that separates collisions (due to, say exchanges) in a sparse array.

The following query computes and store one-minute open/high/low/close bars from these data. We need some extra aggregates from the axial_aggregate plugin: load that:

In [7]:
minute_bars = trades.apply('timeprice', 'tuple(ms,price)')
minute_bars = sdb.afl.regrid(minute_bars, 1000, 1, 60000,
            'axial_first(timeprice) as open',
            'max(price) as high',
            'min(price) as low',
            'axial_last(timeprice) as close'
          )
minute_bars = minute_bars.slice('dummy', 0)\
                            .eval()


In [8]:
minute_bars.todataframe().head()

open    high     low  close
symbol_index ms                              
0            570  61.0  63.500  60.025   62.8
             571  62.7  63.500  62.200   63.5
             572  63.6  64.899  63.000   63.6
             573  63.5  64.500  63.500   64.3
             574  64.1  64.400  64.000   64.4